# Import necessary libraries

In [1]:
import pandas as pd

# Load The MTN Upsell dataset

In [2]:
df = pd.read_excel('../data/raw/mtn_upsell_uncleaned.xlsx')

In [28]:
df.isnull().sum()

dates                   0
Phone Number            0
total_reloads           0
total_reload_amount    68
imei                    0
brand_name              0
model_name              0
device_category         0
data_kb                 0
dtype: int64

# Data Cleaning

As you see we have 60+ nan rows in 'total_reload_amount' column. Common causes are

1. System recording error – Reload was counted, but the amount wasn’t saved.

2. Free bonus/airdrop – A “reload” without money involved (rare).

3. Incomplete data – Some recharge transactions didn’t sync fully.

So i think it's an incomplete data problem, i decide to impute those nan values by the mean of people who reloaded 1 time

In [5]:
mean_total_reloads_amount = df[(df['total_reloads'] == 1) & (df['total_reload_amount'].notna())]['total_reload_amount'].mean()
float(mean_total_reloads_amount)

341.4826373038516

In [6]:
df['total_reload_amount'] = df['total_reload_amount'].fillna(mean_total_reloads_amount)

In [7]:
print(f"Imputed missing values using mean: {mean_total_reloads_amount:.2f}")

Imputed missing values using mean: 341.48


#### Let's find upgraded date for each user

In [13]:
# convert dates columns to datetime
df['dates'] = pd.to_datetime(df['dates'], format='%Y%m%d')

# Sort the DataFrame by 'Phone Number' and 'dates'
df = df.sort_values(by=['Phone Number', 'dates'])


Identify smartphones and those that aren't

In [15]:
df['isSmartphone'] = df['device_category'] == 5


For each user, find the first date they used a smartphone

In [ ]:
dates_upgraded = df[df['isSmartphone']].groupby('Phone Number')['dates'].min().reset_index()
dates_upgraded = dates_upgraded.rename(columns={'dates': 'dates_upgraded', 'Phone Number': 'user'})

Let's get users who used feature/basic phone before

In [38]:
feature_or_basic = df[df['device_category'].isin([4,7])]['Phone Number'].unique()

feature_or_basic

array([225798765432, 225798765433, 225798765434, 225798765435,
       225798765436, 225798765437, 225798765438, 225798765439,
       225798765440, 225798765441, 225798765442, 225798765443,
       225798765444, 225798765445, 225798765447, 225798765448,
       225798765449, 225798765450, 225798765451, 225798765452,
       225798765453, 225798765454, 225798765455, 225798765456,
       225798765457, 225798765458, 225798765459, 225798765460,
       225798765461, 225798765462, 225798765463, 225798765464,
       225798765465, 225798765466, 225798765467, 225798765468,
       225798765469, 225798765470, 225798765471, 225798765472,
       225798765473, 225798765474, 225798765475, 225798765476,
       225798765477, 225798765478, 225798765479, 225798765480,
       225798765481, 225798765482, 225798765483, 225798765484,
       225798765485, 225798765486, 225798765487, 225798765488,
       225798765489])

Filter dates_upgraded to include only those who had a basic/feature phone first

In [39]:
dates_upgraded =  dates_upgraded[dates_upgraded['user'].isin(feature_or_basic)]

Store the dates_upgraded DataFrame in a CSV file

In [ ]:
dates_upgraded.to_csv('../data/cleaned/users_and_date_upgraded.csv', index=False)

#### Build Usage & Recharge Features

For each user in dates_upgraded determine

1. Data used before upgrade => dt_before
2. Data used after upgrade => dt_after
3. Standard deviation of data usage before & after => std_before & std_after
4. Days active after the upgrade where data_kb > 0
5. Recharge growth determined by total_reload_amount (after)-(before)
6. Days passed to first data use = Days between upgrade and first data_kb>0

why standard deviation before & after ?:  This will tell us how consistent or unstable user's data usage behaviour is

> low std : User has a steady behaviour

> high std :  User has unpredicatble behaviour

In short:

I'm not just measuring how much data someone used, but how stable or risky their behavior was.

##### Merge date_upgraded with the original df

In [51]:
#Firstly, we need to rename the 'Phone Number' column to 'user' in the original DataFrame
df = df.rename(columns={'Phone Number': 'user'})

df = df.merge(dates_upgraded, on='user', how='left')

Create columns to label rows as "before" or "after" upgrade

In [74]:
df['Days_to_Upgrade'] = (df['dates']- df['dates_upgraded']).dt.days

df['isBefore'] = df['Days_to_Upgrade'].between(-30, -1) #30 days before upgrade
df['isAfter'] = df['Days_to_Upgrade'].between(0, 60) #60 days after upgrade

df[(df['isBefore'] | df['isAfter'])].head()
   

,dates,user,total_reloads,total_reload_amount,imei,brand_name,model_name,device_category,data_kb,isSmartphone,dates_upgraded,Days_to_Upgrade,isBefore,isAfter
4,2024-09-08,225798765432,1,50.0,1654054197,TECNO,T528,4,0.0,False,2024-10-08,-30,True,False
5,2024-09-11,225798765432,2,450.0,2065123347,TECNO,T528,4,0.0,False,2024-10-08,-27,True,False
6,2024-09-15,225798765432,1,200.0,1441878202,TECNO,T349,4,0.0,False,2024-10-08,-23,True,False
7,2024-09-16,225798765432,3,600.0,1622164687,TECNO,T528,4,0.0,False,2024-10-08,-22,True,False
8,2024-09-17,225798765432,1,150.0,1358567090,TECNO,T528,4,0.0,False,2024-10-08,-21,True,False


Aggregate usage stats per user for both periods

1. before upgrade

In [73]:
before = df[df['isBefore']].groupby('user').agg(
    avg_data_before_upgrade=('data_kb', 'mean'),
    std_before=('data_kb', 'std'),
    total_recharge_before=('total_reload_amount', 'sum')
).reset_index()

before.head()


,user,avg_data_before_upgrade,std_before,total_recharge_before
0,225798765432,0.130697,0.452746,3091.482637
1,225798765435,0.261393,0.452746,450.000000
2,225798765436,7.143986,32.579923,13800.000000
3,225798765438,16.537679,57.288201,4365.000000
4,225798765439,0.000000,0.000000,300.000000


2. after upgrade

In [72]:
after = df[df['isAfter']].groupby('user').agg(
    avg_data_after_upgrade=('data_kb', 'mean'),
    std_after=('data_kb', 'std'),
    total_recharge_after=('total_reload_amount', 'sum'),
    days_active_after=('data_kb', lambda x: (x > 0).sum()),
    time_to_first_data_use=('Days_to_Upgrade', lambda x: x[df.loc[x.index, 'data_kb'] > 0].min())
).reset_index()

after.head()

,user,avg_data_after_upgrade,std_after,total_recharge_after,days_active_after,time_to_first_data_use
0,225798765432,282355.658691,746850.094482,10600.000000,14,0
1,225798765433,390674.369565,760990.974794,15979.550000,10,0
2,225798765434,77583.160156,109719.157305,275.000000,1,0
3,225798765435,156755.774613,284752.858237,25581.482637,26,0
4,225798765436,42745.234739,113248.911365,32883.892637,23,0


3. Merge them together

In [77]:
features = before.merge(after, on='user', how='outer')
features.head()

,user,avg_data_before_upgrade,std_before,total_recharge_before,avg_data_after_upgrade,std_after,total_recharge_after,days_active_after,time_to_first_data_use
0,225798765432,0.130697,0.452746,3091.482637,282355.658691,746850.094482,10600.000000,14,0
1,225798765433,NaN,NaN,NaN,390674.369565,760990.974794,15979.550000,10,0
2,225798765434,NaN,NaN,NaN,77583.160156,109719.157305,275.000000,1,0
3,225798765435,0.261393,0.452746,450.000000,156755.774613,284752.858237,25581.482637,26,0
4,225798765436,7.143986,32.579923,13800.000000,42745.234739,113248.911365,32883.892637,23,0


Add Recharge Growth & Finalize

In [78]:
features['recharge_growth'] = features['total_recharge_after'] - features['total_recharge_before']

## Analysis new features dataframe

In [83]:
features.isna().sum()

user                        0
avg_data_before_upgrade    25
std_before                 34
total_recharge_before      25
avg_data_after_upgrade      0
std_after                   0
total_recharge_after        0
days_active_after           0
time_to_first_data_use      0
recharge_growth            25
dtype: int64

**We have nan values which explain some behaviours of users**

**avg_data_before_upgrade:**  25 user didn’t use mobile data at all before upgrading

**std_before:** no variation because no data

**total_recharge_before:** no top-ups before upgrade , new users or they were inactive

**recharge_growth:** makes sense because it needs both before & after

> Note: 25 users have no data or recharge history before upgrading. Their missing values reflect real user inactivity

***Question:*** why 34 nan values in std_before?

> std_before needs at least 2 data points to calculate a standard deviation. Those extra 9 users (34 NaNs in std_before vs. 25 in avg_data_before_upgrade)  had only one day of data in the 30-day window before their upgrade.

# save the cleaned / engineered dataframe as smartphone_users_features.csv

In [84]:
features.to_csv('../data/cleaned/smartphone_users_features.csv', index=False)